## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

 Импорт библиотек

In [14]:
import pandas as pd # <импорт библиотеки pandas>

Прочитаем файл *data.csv* и сохраним его в переменной *data*.

In [15]:
data = pd.read_csv('/datasets/data.csv') # чтение файла с данными с сохранением в data

Получение первых 10 строк таблицы.

In [16]:
data.head(10) # получение первых 10 строк таблицы data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Общая информация о данных таблицы *data*.

In [17]:
data.info() # получение общей информации о данных в таблице data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Рассмотрим полученную информацию подробнее.

Всего в таблице 12 столбцов, 2 столбца с вещественным типом данных - *float*,
5 столбцов с целочисленным типом данных - *int*, 5 столбцов с типом данных - *object*

Подробно разберём, какие в *data* столбцы и какую информацию они содержат:
* *children* — количество детей в семье
* *days_employed* — общий трудовой стаж в днях
* *dob_years* — возраст клиента в годах
* *education* — уровень образования клиента
* *education_id* — идентификатор уровня образования
* *family_status* — семейное положение
* *family_status_id* — идентификатор семейного положения
* *gender* — пол клиента
* *income_type* — тип занятости
* *debt* — имел ли задолженность по возврату кредитов
* *total_income* — ежемесячный доход
* *purpose* — цель получения кредита

некоторые из названий столбцов лучше заменить на более подходящие, но делать это не обязательно

Количество значений в столбцах различается. Это говорит о том, что в данных есть пропущенные значения.

### Вывод

Каждая строка таблицы содержит обезличенную информацию о клиенте банка (каличество детей, трудовой стаж, возраст, образование, семейный статус, пол, тип занятости, ежемесячный доход и цель взятия кредита).Для проверки гипотезы о влиянии семейного положения и количества детей клиента на факт погашения кредита в срок будут использоваться главным образом столбцы *'children'*, *'family_status'*, *'debt'*. В таблице данных есть пропуски. В столбце *'education'* одинаковые значения имеют различный регистр букв. В  столбце общего стажа работы - *'days_employed'* для ещё работающих клиентов стаж отрицательный. Отрицательные значения возникают из-за формулы расчёта общего стажа работы, который рассчитывается как разность: из суммы дат конца каждого периода трудовой деятельности вычитают сумму дат начала каждого периода трудовой деятельности. Точное значение общего стажа по этой формуле определяется только для пенсионенов, так как для ещё работающих клиентов в формуле не достаёт одного значения.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [18]:
# проверим данные на наличие пропусков вызовом набора методов для суммирования пропущенных значений.
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Видим, что пропуски имеются только в двух столбцах: общий стаж работы - *'days_employed'* и месячный доход - *'total_income'* и их количество одинаковое (скорее всего, для клиентов у которых неизвестен стаж работы, так же неизвестен и месячный доход). Обе переменные являются количественными. Заполним пропуски в таблице в зависимости от типа занятости.

In [19]:
# посмотрим список уникальных значений для столбца с типом занятости и убедимся в отсутствии дубликатов в нём
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [20]:
# заполняем пропуски в общем стаже работы - 'days_employed' в зависимости от типа занятости - 'income_type' 
data['days_employed'].fillna(data.groupby('income_type')['days_employed'].transform('median'), inplace=True)
# заполняем пропуски в ежемесячном доходе - 'total_income' в зависимости от типа занятости - 'income_type' 
data['total_income'].fillna(data.groupby('income_type')['total_income'].transform('median'), inplace=True)
# проверим данные на наличие пропусков вызовом набора методов для суммирования пропущенных значений.
data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Проверим, есть ли в таблице ошибки связанные с возрастом клиентов:

In [21]:
print(data[data['dob_years'] == 0]['dob_years'].count())

101


Так как заменить нулевые значения в столбце возраст клиентов на иные значения корректно непредставляется возможным, и доля строк с нулевыми значениями возраста от общего количества строк менее 0.5%, для этого отфильтруем таблицу используя логическую индексацию.

In [22]:
data = data[data['dob_years'] != 0] # фильтрация таблицы
print(data[data['dob_years'] == 0]['dob_years'].count()) # проверка таблицы после фильтрации

0


### Вывод

Существующие в таблице явные пропуски были заполнены. Строки с нулевым значением для возраста клиентов были удалены.

### Замена типа данных

Столбецы со значениями общего трудового стажа и ежемесячным доходом имеют тип данных *'float'*. Для более наглядного представления данных в таблице и из-за отсутствия физического смысла в столь точных значениях в этих столбцах, тип данных в них стоит заменить на *'int'*.

In [23]:
# переводим все числа в столбце общий трудовой стаж в тип данных int
data['days_employed'] = data['days_employed'].astype('int')
# переводим все числа в столбце ежемесячный доход в тип данных int
data['total_income'] = data['total_income'].astype('int')
# вновь получим общую информацию о типе данных в таблице и убедимся в замене типа данных в нужных столбцах:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21424 entries, 0 to 21524
Data columns (total 12 columns):
children            21424 non-null int64
days_employed       21424 non-null int64
dob_years           21424 non-null int64
education           21424 non-null object
education_id        21424 non-null int64
family_status       21424 non-null object
family_status_id    21424 non-null int64
gender              21424 non-null object
income_type         21424 non-null object
debt                21424 non-null int64
total_income        21424 non-null int64
purpose             21424 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Вывод

Были заменены данные типа *flot* (вещественный числа) на *int* (целые числа) методом *astype*.

### Обработка дубликатов

Приведём символы в столбце образование 'education' к нижнему регистру, так как именно эта проблема для этого столбца была замечена при просмотре первых строк таблицы данных *'data'*

In [24]:
# приводим символы в столбце образование - *'education'* к нижнему регистру
data['education'] = data['education'].str.lower()

In [25]:
# посмотрим список уникальных значений для столбца с семейным статусом и убедимся в отсутствии дубликатов в нём
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [26]:
# хотя в столбце отсутствуют неявные дубликаты, всёже приведём все символы к нижнему регистру в столбце - *'family_status'*
data['family_status'] = data['family_status'].str.lower()

In [27]:
# удалим из таблицы данных data дубликаты
data = data.drop_duplicates().reset_index(drop= True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21353 entries, 0 to 21352
Data columns (total 12 columns):
children            21353 non-null int64
days_employed       21353 non-null int64
dob_years           21353 non-null int64
education           21353 non-null object
education_id        21353 non-null int64
family_status       21353 non-null object
family_status_id    21353 non-null int64
gender              21353 non-null object
income_type         21353 non-null object
debt                21353 non-null int64
total_income        21353 non-null int64
purpose             21353 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [28]:
# посмотрим распределение значений для столбца с целями кредита
data['purpose'].value_counts()

свадьба                                   786
на проведение свадьбы                     764
сыграть свадьбу                           760
операции с недвижимостью                  672
покупка коммерческой недвижимости         658
покупка жилья для сдачи                   649
операции с коммерческой недвижимостью     648
операции с жильем                         646
жилье                                     640
покупка жилья                             640
покупка жилья для семьи                   637
строительство собственной недвижимости    633
недвижимость                              629
операции со своей недвижимостью           627
строительство жилой недвижимости          621
покупка своего жилья                      619
строительство недвижимости                619
покупка недвижимости                      618
ремонт жилью                              605
покупка жилой недвижимости                603
на покупку своего автомобиля              502
заняться высшим образованием      

### Вывод

Одна из причин возникновения дубликатов в таблице: следствие того, что пропуенные значения в столбцах 'общий трудовой стаж' и 'ежемесячный доход' были заполнены медианой по соответствующим группам занятости. Остаётся удалить смысловые дубликаты в столбце с целями кредита

### Лемматизация

Проведём лемматизацию для значений в столбце целей кредита и добавим к таблице новый столбец 

In [29]:
# импорт библиотеки для лемматизации
from pymystem3 import Mystem
m = Mystem()

Напишем функцию для лемматизации строк в столбце, которая возвращает значение цели кредита: жильё, свадьба, недвижемость, автомобиль, образование. С её помощью заполним столбец с лемматизированными целями кредита.

In [30]:
# напишем функцию для лемматизации строк в столбце целей кредита
def lemma_purpose_text(row):
    lemmas = m.lemmatize(row)
    if 'свадьба' in lemmas:
        return 'свадьба'
    elif 'жилье' in lemmas:
        return 'жилье'
    elif 'недвижимость' in lemmas:
        return 'недвижимость'
    elif 'автомобиль' in lemmas:
        return 'автомобиль'
    elif 'образование' in lemmas:
        return 'образование'
# функция на одной строке для лемматизации столбца целей кредита в таблице данных
data['lemma_purpose'] = data['purpose'].apply(lemma_purpose_text)

Проверим, что новый столбец не содержит пропусков

In [31]:
data['lemma_purpose'].isnull().sum()

0

### Вывод

С помощью ламмизации целей взятия кредитов клиентами из таблицы данных были выделены соответствующие леммы целей (свадьба, жильё, недвижимость, автомобиль, образование)

### Категоризация данных

In [32]:
# посмотрим распределение значений для столбца с количеством детей в семье
data['children'].value_counts()

 0     14022
 1      4792
 2      2039
 3       328
 20       75
-1        47
 4        41
 5         9
Name: children, dtype: int64

Значение 20 и -1 явно аномальные, эти ошибки могли возникнуть при записи информации в таблицу. Заменим 20 на 2, а -1 на 1.

In [33]:
data['children'] = data['children'].replace(-1, 1).replace(20, 2)
data['children'].value_counts()

0    14022
1     4839
2     2114
3      328
4       41
5        9
Name: children, dtype: int64

In [34]:
data['family_status'].value_counts() # проверили столбец на наличие ошибок

женат / замужем          12290
гражданский брак          4130
не женат / не замужем     2794
в разводе                 1185
вдовец / вдова             954
Name: family_status, dtype: int64

In [35]:
# выясним id каждого семейного статуса
family_status_id = data[['family_status','family_status_id']]
family_status_id.drop_duplicates().reset_index(drop= True)

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


In [36]:
data['debt'].value_counts() # проверили столбец на наличие ошибок

0    19620
1     1733
Name: debt, dtype: int64

В столбце *'dept'* 0 - означает отсутствие проблем с возвратом кредита, а 1 - то что имелись проблемы с возвратом кредита

Распределим клиентов по категориям в зависимости от количества детей:
Напишем функцию, которая возвращает категорию семьи в зависимости от количества детей, используя правило:
* 0 детей - бездетные
* 1 ребёнок- однодетные
* 2 ребёнка- малодетные
* детей более 2 - многодетные

In [37]:
def family_categories(children):
    if children == 0:
        return 'бездетная'
    if children == 1:
        return 'однодетная'
    if children == 2:
        return 'малодетная'
    return 'многодетная'
# добавим к таблице новый столбец со значениями категории семьи в зависимости от количества детей
data['children_family_status'] = data['children'].apply(family_categories)

### Вывод

Категоризацию проводили по числу детей в семье, так как в большинстве случаев у клиентов нет детей, либо их 1 - 2, и лишь небольшой процент клиентов имеют многодетные семьи

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Сгруппируем семьи по категории количества детей, посчитаем общее число заёмщиков и количество случаев проблем с возвратом кредита. Затем посчитаем процент возникновения проблем по каждой группе.

In [38]:
log_groupby = data.groupby('children_family_status').agg({'debt': ['count', 'sum']})
log_groupby['percentage_of_problem_cases_%'] = log_groupby['debt']['sum'] / log_groupby['debt']['count'] * 100
log_groupby

debt       percentage_of_problem_cases_%
                        count   sum                              
children_family_status                                           
бездетная               14022  1058                      7.545286
малодетная               2114   202                      9.555345
многодетная               378    31                      8.201058
однодетная               4839   442                      9.134119

### Вывод

Процент проблем с возвратом кредитов наименьший для бездетных клиентов. Примерно однинаковый процент проблем с возвратом кредитов приходится на клиентов с детьми. Для многодетных семей этот показатель меньше чем для однодетных и малодетных, но и количество многодетных семей на порядок меньше.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Сгруппируем клиентов по уровню доходов, посчитаем общее число заёмщиков и количество случаев проблем с возвратом кредита. Затем посчитаем процент возникновения проблем по каждой группе.

In [39]:
log_groupby1 = data.groupby('family_status').agg({'debt': ['count', 'sum']})
log_groupby1['percentage_of_problem_cases_%'] = log_groupby1['debt']['sum'] / log_groupby1['debt']['count'] * 100
log_groupby1

debt      percentage_of_problem_cases_%
                       count  sum                              
family_status                                                  
в разводе               1185   85                      7.172996
вдовец / вдова           954   62                      6.498952
гражданский брак        4130  386                      9.346247
женат / замужем        12290  927                      7.542718
не женат / не замужем   2794  273                      9.770938

### Вывод

Наибольший процент проблем с возвратом кредитов наблюдается для клиентов, которые живут гражданским браком (сожительство) или пока ещё не имеюие семьи.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [40]:
print('минимальный ежемесячный доход клиента:', data['total_income'].min())
print('максимальный ежемесячный доход клиента', data['total_income'].max())
print('медиана ежемесячных доходом клиентов', data['total_income'].median())

минимальный ежемесячный доход клиента: 20667
максимальный ежемесячный доход клиента 2265604
медиана ежемесячных доходом клиентов 142594.0


Распределим клиентов по категориям в зависимости от доходов: напишем функцию, которая возвращает категорию клиента в зависимости от количества детей, используя правило:
* доход:
    * от 20000 до 30000 - выше бедности
    * от 30000 до 60000 - средний достаток
    * от 60000 до 90000 - состоятельные
    * от 90000 до 150000 - богатые
    * от 150000 и выше - сверхбогатые

In [41]:
def standard_of_living(total_income):
    if total_income >= 20000 and total_income < 30000:
        return 'выше бедности'
    if total_income >= 30000 and total_income < 60000:
        return 'средний достаток'
    if total_income >= 60000 and total_income < 90000:
        return 'состоятельные'
    if total_income >= 90000 and total_income < 150000:
        return 'богатые'
    return 'сверхбогатые'
# добавим к таблице новый столбец со значениями категории семьи в зависимости от количества детей
data['standard_of_living'] = data['total_income'].apply(standard_of_living)

Сгруппируем клиентов по уровню доходов, посчитаем общее число заёмщиков и количество случаев проблем с возвратом кредита. Затем посчитаем процент возникновения проблем по каждой группе.

In [42]:
log_groupby2 = data.groupby('standard_of_living').agg({'debt': ['count', 'sum']})
log_groupby2['percentage_of_problem_cases_%'] = log_groupby2['debt']['sum'] / log_groupby2['debt']['count'] * 100
log_groupby2

debt      percentage_of_problem_cases_%
                   count  sum                              
standard_of_living                                         
богатые             8239  712                      8.641825
выше бедности         22    2                      9.090909
сверхбогатые        9783  759                      7.758356
состоятельные       2529  213                      8.422301
средний достаток     780   47                      6.025641

### Вывод

Люди со среднем достатком стараются не иметь проблем по выплате кредитов. Наиболее рискованную группу представляют собой клиенты с доходами до 30000р, но здесь сложно обобщать из-за малого числа клиентов данного типа.

- Как разные цели кредита влияют на его возврат в срок?

Сгруппируем клиентов по целям кредита, посчитаем общее число заёмщиков и количество случаев проблем с возвратом кредита. Затем посчитаем процент возникновения проблем по каждой группе.

In [43]:
log_groupby3 = data.groupby('lemma_purpose').agg({'debt': ['count', 'sum']})
log_groupby3['percentage_of_problem_cases_%'] = log_groupby3['debt']['sum'] / log_groupby3['debt']['count'] * 100
log_groupby3

debt      percentage_of_problem_cases_%
              count  sum                              
lemma_purpose                                         
автомобиль     4284  400                      9.337068
жилье          4436  306                      6.898106
недвижимость   6328  473                      7.474716
образование    3995  370                      9.261577
свадьба        2310  184                      7.965368

### Вывод

Клиенты, которые берут кредит с целью приобретения жилья или операций с ним стараются выплачивать кредит вовремя. Наибольший процент проблем с выплотой кредитов встречается у клиентов, которые берут кредит для оплаты образованию или покупки авто.

### Шаг 4. Общий вывод

Проверка четырёх гипотез о зависимости числа проблем выплат кредита от ряда параметров количества детей в семье, семейного статуса, дохода и целей кредита клиентов. Разброс значения процента возникновения проблем с выплотами к общему числу случаев невелик (в переделах 3-4%).C повышением числа детей в семье растут риски невыплат по кредитам. Клиенты у которых есть семьи или она была у них в прошлом демонстрируют более высокий уровень ответственности по своим обязательствам. Люди с доходами выше уровня бедности находятся в зоне риска по невыплатам за кредиты. Покупка жилья для клиентов является жизненной необходимостью, поэтому риски по невыплотам гораздо ниже, чем за приобретение предметов роскоши (автомобиль или свадьба) или рискованных инвестицей (образование).